In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pandas as pd
from tqdm.autonotebook import tqdm
from utils.elasticsearch_utils import ElasticSearchUtils

/home/.local/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
host_info = {
    'host': 'https://corpus.ncsoft.com:9200',
    'http_auth': 'elastic:nlplab',
}

utils = ElasticSearchUtils(**host_info)

In [ ]:
index_list = [
    'corpus-bitext-cnn',
    'corpus-bitext-donga',
    'corpus-bitext-kusk',
    'corpus-bitext-ted',
]

result = []
for index in tqdm(index_list):
    utils.export(index=index, result=result)
    
df = pd.DataFrame(result)    

In [ ]:
df = pd.read_json(
    'clean-bitext.morp.json.bz2',
    compression='bz2',
    orient='records',
    lines=True,
)

In [ ]:
df.head()

In [ ]:
df.groupby(by='source').size().to_frame()

In [ ]:
def remove_borken_code(df):
    borken_code = df[ df['korean'].str.contains('') ]
    borken_code = df[ df['korean'].str.contains('') ]
    borken_code = df[ df['korean'].str.contains('ë') ]
    borken_code
    
    df.drop(borken_code.index, inplace=True)
    
    return

In [ ]:
def split_train_test(result_df, df):
    random_seed = 10

    train_df = df.sample(frac=0.9, random_state=random_seed)
    test_df = df.drop(train_df.index)
    
    result_df.at[train_df.index, 'role'] = 'train'
    result_df.at[test_df.index, 'role'] = 'test'
    
    return

def split_data_set(df):
    split_train_test(result_df=df, df=df[df['source'] == 'cnn'])
    split_train_test(result_df=df, df=df[df['source'] == 'donga'])
    split_train_test(result_df=df, df=df[df['source'] == 'kusk'])
    split_train_test(result_df=df, df=df[df['source'] == 'ted'])
    
    return df

In [ ]:
df = split_data_set(df)

df.groupby(by=['type', 'role']).size().to_frame()

In [ ]:
df.to_json(
    'clean-bitext.morp.clean.json.bz2',
    force_ascii=False,
    compression='bz2',
    orient='records',
    lines=True,
)

In [ ]:
df[df['role'] == 'train'].to_csv('clean-bitext.morp.train.csv', sep='\t', header=False)
df[df['role'] == 'test'].to_csv('clean-bitext.morp.test.csv', sep='\t', header=False)

In [7]:
df = pd.read_csv('clean-bitext.morp.test.csv', sep='\t', header=None)
df

,0,1,2,3,4,5,6,7
0,2,cnn,There are no plans for that delegation to trav...,로버트 우드 미 국무부 부대변인은 5일 “보즈워스 특별대표의 북한 방문 일정은 없다...,로버트/NNP 우드/NNP 미/NNP 국무부/NNG 부대변인/NNG+은/JX 5/S...,cnn,cnn-10006,test
1,23,cnn,But Rooney's arrival led to a frenzied spell o...,그러나 루니의 교체투입으로 후반 18분을 남겨놓고 6골을 넣는 화려한 난타전이 벌어...,그러나/MAJ 루니/NNP+의/JKG 교체/NNG+투입/NNG+으로/JKB 후반/N...,cnn,cnn-10064,test
2,26,cnn,At first Curtis Christiansen figured he was ti...,처음엔 커티스 크리스티안센은 일에 지쳐 피곤했다고 생각했다. 그는 코골기를 피곤하다...,처음/NNG+에/JKB+ㄴ/JX 커티스/NNP 크리스티안센/NNP+은/JX 일/NN...,cnn,cnn-10075,test
3,35,cnn,"There's going to be a natural reaction,"" the p...",그는 “주식시장의 이와 같은 반응은 당연하다”며 “장기적인 관점에서 보면 지금이 수...,그/NP+는/JX “/SW+주식/NNG+시장/NNG+의/JKG 이/NP+와/JKB ...,cnn,cnn-10099,test
4,50,cnn,Those are just a few of more than 60 restricti...,이는 중국이 지난해 제한한 62차례 보도규제에 대해 국제기자연맹(IFJ)측의 보고 ...,이/NP+는/JX 중국/NNP+이/JKS 지난해/NNG 제한/NNG+하/XSV+ㄴ/...,cnn,cnn-10142,test
5,62,cnn,This meant Fink's invention was useful not onl...,핑크 교수의 완벽한 거울 발명은 군사분야와 원격통신뿐만 아니라 최소침습적 미세 레이...,핑크/NNG 교수/NNG+의/JKG 완벽/NNG+하/XSA+ㄴ/ETM 거울/NNG ...,cnn,cnn-10174,test
6,72,cnn,"Schapiro, who will be the first woman to chair...",샤피로 의원은 상원의 승인을 받게 되면 미국 최초의 SEC 위원장이 된다. 그가 최...,샤피로/NNP 의원/NNG+은/JX 상원/NNG+의/JKG 승인/NNG+을/JKO ...,cnn,cnn-10197,test
7,86,cnn,The triple Olympic gold medallist fought off c...,베이징 올림픽 금메달 3관왕인 크리스 호이는 자동차 경주대회 포뮬라 1 의 세계 챔...,베이징/NNP 올림픽/NNG 금메달/NNG 3/SN+관왕/NNG+이/VCP+ㄴ/ET...,cnn,cnn-10237,test
8,91,cnn,And space-inspired fashion is already making c...,또한 우주에 영감을 받은 패션은 이미 디자인계에 우주적 물결이 밀려들고 있다.,또한/MAG 우주/NNG+에/JKB 영감/NNG+을/JKO 받/VV+은/ETM 패션...,cnn,cnn-10249,test
9,135,cnn,CNN) -- Seven miners were killed Monday by an ...,미국 웨스트버지니아주 랄리카운티에 있는 탄광에서 5일(현지시간) 폭발사고가 일어나 ...,미국/NNP 웨스트버지니아주/NNP 랄리카운티/NNP+에/JKB 있/VV+는/ETM...,cnn,cnn-10372,test
